In [ ]:
import networkx as nx
from collections import Counter

# Load the karate club graph (NetworkX has this built-in)
karate = nx.karate_club_graph()

# Find all maximal cliques
clique_out = list(nx.find_cliques(karate))

# Get the number of cliques
num_cliques = len(clique_out)
print(f"Number of cliques: {num_cliques}")

# Get the size distribution of cliques
clique_sizes = [len(clique) for clique in clique_out]
size_counts = Counter(clique_sizes)
print("Clique size distribution:")
for size, count in sorted(size_counts.items()):
    print(f"Size {size}: {count} cliques")

# Find all cliques of size 5
cliques_of_size_5 = [clique for clique in clique_out if len(clique) == 5]
print(f"\nCliques of size 5: {cliques_of_size_5}")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# -------------------------------------------------
# 1. Load or create a social network
# -------------------------------------------------
# Use the Karate Club network (classic social network with known split)
G = nx.karate_club_graph()

# Make sure it's undirected and simple
G = nx.Graph(G)

# -------------------------------------------------
# 2. For each node, count local 3-node motifs:
#    - Triangles (closed triads)
#    - Open triads (structural holes)
# -------------------------------------------------
def compute_local_motifs(G):
    """
    Returns a dict with for each node:
      - 'triangles': number of triangles the node participates in
      - 'open_triads': number of open triads centered at the node
    """
    motif_data = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        n_neighbors = len(neighbors)
        
        # Number of edges among neighbors = triangles centered at node
        subgraph = G.subgraph(neighbors)
        edges_among_neighbors = subgraph.number_of_edges()
        
        # Triangles = each edge among neighbors forms a triangle with node
        triangles = edges_among_neighbors
        
        # Open triads = all possible pairs of neighbors minus actual edges
        # Total possible pairs = C(n,2)
        total_pairs = n_neighbors * (n_neighbors - 1) // 2
        open_triads = total_pairs - edges_among_neighbors
        
        motif_data[node] = {
            'triangles': triangles,
            'open_triads': open_triads,
            'degree': n_neighbors
        }
    return motif_data

motif_data = compute_local_motifs(G)

# Convert to DataFrame for easy analysis
df = pd.DataFrame.from_dict(motif_data, orient='index')
df.index.name = 'node'
df = df.sort_index()

print("Motif counts for first 10 nodes:")
print(df.head(10))

# -------------------------------------------------
# 3. Build a "Motif-Based Influencer Score"
# -------------------------------------------------
# Idea: Balance between being well-connected (triangles) and bridging (open_triads)
# We normalize both metrics to [0,1] and combine them.

# Normalize using min-max (robust for small networks)
df['triangles_norm'] = df['triangles'] / df['triangles'].max() if df['triangles'].max() > 0 else 0
df['open_triads_norm'] = df['open_triads'] / df['open_triads'].max() if df['open_triads'].max() > 0 else 0

# Influencer Score = geometric mean (penalizes extreme imbalance)
# Alternatively: arithmetic mean, or weighted (e.g., 0.4*tri + 0.6*open)
df['influencer_score'] = np.sqrt(df['triangles_norm'] * df['open_triads_norm'])

# Handle nodes with no neighbors
df['influencer_score'] = df['influencer_score'].fillna(0)

# Sort by score
df_sorted = df.sort_values('influencer_score', ascending=False)
print("\nTop 5 Influencers by Motif Score:")
print(df_sorted[['degree', 'triangles', 'open_triads', 'influencer_score']].head())

# -------------------------------------------------
# 4. Visualize the network with influencer scores
# -------------------------------------------------
# Map score to color and size
node_colors = [df.loc[node, 'influencer_score'] for node in G.nodes()]
node_sizes = [500 * (df.loc[node, 'influencer_score'] + 0.1) for node in G.nodes()]  # +0.1 for visibility

plt.figure(figsize=(12, 9))
pos = nx.spring_layout(G, seed=42)  # fixed layout

nx.draw_networkx_edges(G, pos, alpha=0.5)
nx.draw_networkx_nodes(
    G, pos,
    node_color=node_colors,
    node_size=node_sizes,
    cmap=plt.cm.viridis,
    edgecolors='k',  # black border for clarity
    linewidths=0.5
)
nx.draw_networkx_labels(G, pos, font_size=8)

plt.title("Social Network: Motif-Based Influencer Score\n(Color & Size = Influencer Score)")
plt.axis('off')
plt.tight_layout()
plt.show()

# -------------------------------------------------
# 5. Optional: Compare with traditional centrality
# -------------------------------------------------
df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
df['betweenness'] = pd.Series(nx.betweenness_centrality(G))

print("\nCorrelation with traditional metrics:")
print(df[['influencer_score', 'degree_centrality', 'betweenness']].corr()['influencer_score'])

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create graph with two cliques
G = nx.Graph()

# Define cliques
clique1 = ['A', 'B', 'C', 'D', 'E']
clique2 = ['F', 'G', 'H', 'I', 'J']

# Add nodes and clique edges
G.add_nodes_from(clique1 + clique2)
G.add_edges_from([(u, v) for i, u in enumerate(clique1) for v in clique1[i+1:]])  # Clique 1 edges
G.add_edges_from([(u, v) for i, u in enumerate(clique2) for v in clique2[i+1:]])  # Clique 2 edges
G.add_edges_from([('A', 'F'), ('B', 'G')])  # Inter-clique edges

# Visualize graph with consistent layout
plt.figure(figsize=(8, 5))
nx.draw(G, pos=nx.spring_layout(G, seed=7, k=1, iterations=50), 
        node_color='lightblue', node_size=500, edge_color='black', 
        font_size=10, font_weight='bold')
plt.title('Two Cliques Graph')
plt.axis('off')
plt.show()

In [ ]:
pip install networkx python-louvain matplotlib leidenalg

In [ ]:
# pip install networkx python-louvain matplotlib leidenalg
import networkx as nx
import community as community_louvain  # This is python-louvain
import matplotlib.pyplot as plt

# 1. Create or load a graph
# We'll use the famous "Karate Club" network (a classic benchmark)
G = nx.karate_club_graph()

# Optional: Add labels (node names) for clarity
# Each node has an attribute 'club' (Mr. Hi or John A)
# We'll use it later for ground-truth comparison

# 2. Detect communities using the Louvain algorithm
partition = community_louvain.best_partition(G)

# partition is a dict: {node: community_id}
print("Number of communities detected:", len(set(partition.values())))
print("Community assignment (first 10 nodes):", {k: partition[k] for k in list(partition)[:10]})

# 3. Compute modularity (Q)
modularity_score = community_louvain.modularity(partition, G)
print(f"Modularity (Q) = {modularity_score:.4f}")

# 4. Visualize the network with community coloring
# Assign a color to each community
color_map = [partition[node] for node in G.nodes()]

plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)  # fixed layout for reproducibility
nx.draw_networkx(
    G,
    pos,
    node_color=color_map,
    cmap=plt.cm.tab20,
    node_size=300,
    with_labels=True,
    font_size=10
)
plt.title(f"Community Detection via Louvain (Modularity Q = {modularity_score:.4f})")
plt.axis("off")
plt.tight_layout()
plt.show()

# Leiden Algorithm (More Robust)
# pip install leidenalg
import leidenalg as la
import igraph as ig

# Convert NetworkX graph to igraph
G_ig = ig.Graph.from_networkx(G)

# Run Leiden
partition_leiden = la.find_partition(G_ig, la.ModularityVertexPartition)
print("Leiden: Number of communities =", len(partition_leiden))

# Modularity is built into the partition
print("Leiden modularity =", partition_leiden.quality())

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def generate_sbm(block_sizes, p_in, p_out, seed=None):
    """
    Generate an undirected SBM graph.
    
    Parameters:
        block_sizes: list of ints, size of each community
        p_in: float, edge probability within communities
        p_out: float, edge probability between communities
        seed: random seed
    Returns:
        G: networkx Graph
        labels: list of true community labels for each node
    """
    if seed:
        np.random.seed(seed)
    
    n_total = sum(block_sizes)
    G = nx.Graph()
    G.add_nodes_from(range(n_total))
    
    # Assign true labels
    labels = []
    for i, size in enumerate(block_sizes):
        labels.extend([i] * size)
    
    # Add edges
    for i in range(n_total):
        for j in range(i + 1, n_total):
            if labels[i] == labels[j]:
                p = p_in
            else:
                p = p_out
            if np.random.rand() < p:
                G.add_edge(i, j)
    
    return G, labels

# Example usage
block_sizes = [50, 50, 50]  # 3 communities of 50 nodes each
G, true_labels = generate_sbm(block_sizes, p_in=0.15, p_out=0.02, seed=42)

print(f"Generated graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

# Visualize (optional; may be messy for large graphs)
pos = nx.spring_layout(G, seed=42)
colors = [f"C{label}" for label in true_labels]
nx.draw(G, pos, node_color=colors, node_size=20, with_labels=False)
plt.title("SBM Graph with 3 Communities")
plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Load a network
G = nx.karate_club_graph()

# Use degree centrality as a proxy for "coreness"
# (You can also try eigenvector_centrality, betweenness, etc.)
coreness = nx.degree_centrality(G)
# Normalize to [0, 1] if desired (degree_centrality is already normalized by max possible degree)

# Visualize
pos = nx.spring_layout(G, seed=42)
colors = [coreness[node] for node in G.nodes()]

plt.figure(figsize=(8, 6))
nx.draw(G, pos, node_color=colors, cmap='coolwarm', node_size=300, with_labels=True)
plt.title("Core–Periphery Structure (Red = Core, Blue = Periphery)")
plt.show()

print("Top 5 core nodes:", sorted(coreness, key=coreness.get, reverse=True)[:5])

In [1]:
import networkx as nx
import random

# Set seed for reproducibility
random.seed(42)

# Initialize graph with 200 nodes
G = nx.Graph()
G.add_nodes_from(range(200))  # 0–99: Pro-Vax, 100–199: Skeptical

# Add dense internal connections within each group
print("Adding intra-group edges...")
for i in range(100):
    for j in range(i + 1, 100):
        if random.random() < 0.08:
            G.add_edge(i, j)

for i in range(100, 200):
    for j in range(i + 1, 200):
        if random.random() < 0.07:
            G.add_edge(i, j)

# Add sparse bridge edges between groups
print("Adding inter-group (bridge) edges...")
for _ in range(8):
    a = random.randint(0, 99)
    b = random.randint(100, 199)
    G.add_edge(a, b)

# Ground-truth persona labels
true_persona = {node: "Pro-Vax" if node < 100 else "Skeptical" for node in G.nodes()}
true_label = {node: 0 if node < 100 else 1 for node in G.nodes()}  # 0 = Pro-Vax, 1 = Skeptical

# Basic network stats
n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()

pro_edges = sum(1 for u, v in G.edges() if u < 100 and v < 100)
skep_edges = sum(1 for u, v in G.edges() if u >= 100 and v >= 100)
bridge_edges = n_edges - pro_edges - skep_edges

print(f"\nNetwork built: {n_nodes} users, {n_edges} interactions")
print(f"  - Pro-Vax internal edges: {pro_edges}")
print(f"  - Skeptical internal edges: {skep_edges}")
print(f"  - Bridge (cross-group) edges: {bridge_edges}")

# Compute modularity of ground-truth partition
from networkx.algorithms import community as nx_comm

partition = [
    [node for node in G.nodes() if true_label[node] == 0],
    [node for node in G.nodes() if true_label[node] == 1]
]
modularity = nx_comm.modularity(G, partition)
print(f"\nModularity of true communities: {modularity:.4f}")

# Compute average Jaccard similarity within each group (as a measure of internal cohesion)
def avg_jaccard_in_group(G, nodes):
    total_sim = 0.0
    count = 0
    node_list = list(nodes)
    for i in range(len(node_list)):
        for j in range(i + 1, len(node_list)):
            u, v = node_list[i], node_list[j]
            neighbors_u = set(G.neighbors(u))
            neighbors_v = set(G.neighbors(v))
            intersection = neighbors_u & neighbors_v
            union = neighbors_u | neighbors_v
            if len(union) > 0:
                jaccard = len(intersection) / len(union)
                total_sim += jaccard
                count += 1
    return total_sim / count if count > 0 else 0.0

pro_nodes = [n for n in G.nodes() if n < 100]
skep_nodes = [n for n in G.nodes() if n >= 100]

avg_jaccard_pro = avg_jaccard_in_group(G, pro_nodes)
avg_jaccard_skep = avg_jaccard_in_group(G, skep_nodes)

print(f"\nAverage Jaccard similarity:")
print(f"  - Within Pro-Vax group: {avg_jaccard_pro:.4f}")
print(f"  - Within Skeptical group: {avg_jaccard_skep:.4f}")

# Optional: store attributes on graph for later use
nx.set_node_attributes(G, true_persona, 'persona')
nx.set_node_attributes(G, true_label, 'label')

print("\nGraph ready for simulation (e.g., message targeting, retention modeling, or community detection).")

Adding intra-group edges...
Adding inter-group (bridge) edges...

Network built: 200 users, 714 interactions
  - Pro-Vax internal edges: 371
  - Skeptical internal edges: 335
  - Bridge (cross-group) edges: 8

Modularity of true communities: 0.4875

Average Jaccard similarity:
  - Within Pro-Vax group: 0.0375
  - Within Skeptical group: 0.0341

Graph ready for simulation (e.g., message targeting, retention modeling, or community detection).


In [2]:
import networkx as nx

# Define community sizes and connection probabilities
sizes = [150, 200, 100]  # Total: 450 nodes
# Block 0: 0–149 → "Home Baristas"
# Block 1: 150–349 → "Busy Professionals"
# Block 2: 350–449 → "Gift Buyers"

prob_matrix = [
    [0.08,  0.01,  0.005],  # Home Baristas
    [0.01,  0.06,  0.008],  # Busy Professionals
    [0.005, 0.008, 0.05]    # Gift Buyers
]

# Generate stochastic block model graph
G = nx.stochastic_block_model(sizes, prob_matrix, seed=2025, directed=False)

# Assign true personas
true_persona = {}
persona_types = ["Home Baristas", "Busy Professionals", "Gift Buyers"]
start = 0
for i, size in enumerate(sizes):
    for node in range(start, start + size):
        true_persona[node] = persona_types[i]
    start += size

# Store persona as node attribute
nx.set_node_attributes(G, true_persona, 'true_persona')

# --- Diagnostics ---
n = G.number_of_nodes()
m = G.number_of_edges()
print(f"Network generated: {n} nodes, {m} edges")

# Count intra- vs inter-block edges
intra_edges = 0
inter_edges = 0
block_id = {}
start = 0
for block_idx, size in enumerate(sizes):
    for node in range(start, start + size):
        block_id[node] = block_idx
    start += size

for u, v in G.edges():
    if block_id[u] == block_id[v]:
        intra_edges += 1
    else:
        inter_edges += 1

print(f"Intra-community edges: {intra_edges}")
print(f"Inter-community edges: {inter_edges}")
print(f"Inter-community edge fraction: {inter_edges / m:.3f}")

# Compute modularity of ground-truth partition
from networkx.algorithms import community as nx_comm
partition = [
    list(range(0, 150)),
    list(range(150, 350)),
    list(range(350, 450))
]
mod = nx_comm.modularity(G, partition)
print(f"Modularity of true communities: {mod:.4f}")

# Optional: compute average Jaccard similarity per community (as proxy for internal cohesion)
def avg_jaccard(G, nodes):
    total, count = 0.0, 0
    node_list = list(nodes)
    for i in range(len(node_list)):
        for j in range(i + 1, len(node_list)):
            u, v = node_list[i], node_list[j]
            N_u = set(G.neighbors(u))
            N_v = set(G.neighbors(v))
            union = N_u | N_v
            if union:
                jacc = len(N_u & N_v) / len(union)
                total += jacc
                count += 1
    return total / count if count > 0 else 0.0

for i, (start, size) in enumerate(zip([0, 150, 350], sizes)):
    nodes = range(start, start + size)
    jacc = avg_jaccard(G, nodes)
    print(f"Average Jaccard similarity in {persona_types[i]}: {jacc:.4f}")

print("\nGraph is ready for persona-aligned simulation.")

Network generated: 450 nodes, 2906 edges
Intra-community edges: 2389
Inter-community edges: 517
Inter-community edge fraction: 0.178
Modularity of true communities: 0.4197
Average Jaccard similarity in Home Baristas: 0.0352
Average Jaccard similarity in Busy Professionals: 0.0273
Average Jaccard similarity in Gift Buyers: 0.0207

Graph is ready for persona-aligned simulation.
